## Import Libraries

In [111]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist,jaccard
import sys,  datetime, os
import requests
import chardet
from cmapPy import pandasGEXpress as pdx

## Load Data

#### Load L1000 Data

In [112]:
gctoo = pdx.parse('Input/CD_signatures_LM_42809x978.gctx')

In [113]:
gctoo.data_df.head()

cid,b'200814_at',b'222103_at',b'201453_x_at',b'204131_s_at',b'200059_s_at',b'205067_at',b'213702_x_at',b'214435_x_at',b'201334_s_at',b'213721_at',...,b'204662_at',b'203047_at',b'203343_at',b'217995_at',b'218450_at',b'212536_at',b'218529_at',b'211071_s_at',b'203341_at',b'205379_at'
rid,,,,,,,,,,,,,,,,,,,,,
b'AML001_CD34_6H:BRD-K43389675:10',0.004217,-0.018644,0.019958,-0.056607,-0.017211,-0.040410,-0.012175,-0.013127,-0.006405,0.060587,...,-0.025667,0.012359,0.028994,0.035868,0.012153,0.036431,0.001324,0.039140,-0.055924,-0.012614
b'AML001_PC3_6H:BRD-A19037878:0.37037',0.000596,-0.085928,-0.012058,0.002132,0.005287,-0.001508,-0.005940,0.030827,0.010825,-0.004869,...,0.021397,-0.025200,-0.019982,-0.000575,0.039860,0.014524,-0.031927,-0.022506,0.015047,-0.066462
b'AML001_PC3_6H:BRD-A19037878:1.11111',0.007447,-0.074915,-0.011149,0.002667,0.012776,-0.006488,0.011783,0.001526,0.006922,0.004500,...,-0.007810,-0.032042,0.010248,-0.035808,0.020633,-0.003173,0.022985,-0.057549,0.049375,-0.173901
b'AML001_PC3_6H:BRD-A19037878:10',0.015352,-0.084914,-0.008794,0.001621,0.005215,0.003876,0.016859,0.022594,0.051042,-0.003008,...,0.008252,-0.028588,-0.047041,-0.016308,0.014576,0.001701,0.003869,-0.050433,-0.023611,-0.127203
b'AML001_PC3_6H:BRD-A19037878:3.33333',0.002953,-0.087428,-0.009396,0.011583,0.002616,0.014878,0.024690,0.001625,0.014958,-0.000704,...,0.024464,-0.019280,-0.042911,-0.020822,0.027215,0.016594,-0.006432,-0.059470,0.021893,-0.062428


In [149]:
pcid_pertid = pd.read_csv('Input/PCID_pertid_mapping.csv')
pcid_pertid.tail()

,pert_id,pubchem_cid
20444,CMAP-ERGKD328,NaN
20445,CMAP-PRISM-1B-NR,NaN
20446,CMAP-PRISM-1B-PARENT,NaN
20447,CMAP-PRISM-TP7,NaN
20448,CMAP-T2DTUNICAMYCIN,NaN


## Get smallest mean cosine dist for each drug

In [115]:
row_meta_df_g = gctoo.row_metadata_df.sort_values(['pert_id', 'mean_cosine_dist_centered_by_batch'])\
    .groupby('pert_id')\
    .head(1)
row_meta_df_g.head()

rhd,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time
rid,,,,,,,,
b'CPC015_PHH_24H:BRD-A00100033-001-04-8:10',0.0203,b'CPC015_PHH_24H',b'PHH',b'0.911062239822',b'Nifurtimox',10.0,b'BRD-A00100033',24.0
b'CPC001_HA1E_24H:BRD-A00267231-001-01-1:10',0.0022,b'CPC001_HA1E_24H',b'HA1E',b'0.897246550352',b'HEMADO',10.0,b'BRD-A00267231',24.0
b'CPC019_MCF7_24H:BRD-A00420644-001-01-7:10',0.0000,b'CPC019_MCF7_24H',b'MCF7',b'0.728396919096',b'-666',10.0,b'BRD-A00420644',24.0
b'CPC010_HA1E_6H:BRD-A00474148-001-02-3:10',0.0007,b'CPC010_HA1E_6H',b'HA1E',b'0.913214172054',b'-666',10.0,b'BRD-A00474148',6.0
b'CPC011_HA1E_6H:BRD-A00520476-001-05-8:10',0.0147,b'CPC011_HA1E_6H',b'HA1E',b'0.890210541177',"b'6h-pyrido[2,3-b][1,4]benzodiazepin-6-one, 11...",10.0,b'BRD-A00520476',6.0


## Make DF

In [145]:
df = pd.DataFrame()
for ind in row_meta_df_g.index:
    to_append = gctoo.data_df.loc[[ind]]
    df = df.append(to_append)
df.tail()

cid,b'200814_at',b'222103_at',b'201453_x_at',b'204131_s_at',b'200059_s_at',b'205067_at',b'213702_x_at',b'214435_x_at',b'201334_s_at',b'213721_at',...,b'204662_at',b'203047_at',b'203343_at',b'217995_at',b'218450_at',b'212536_at',b'218529_at',b'211071_s_at',b'203341_at',b'205379_at'
rid,,,,,,,,,,,,,,,,,,,,,
b'CPC006_HA1E_24H:BRD-U88459701-000-01-8:10',0.011131,0.008376,-0.008477,0.019005,0.065456,0.015523,0.010625,0.014633,0.015544,0.014923,...,0.000764,0.026699,0.021647,0.021657,0.011010,0.039733,-0.013453,0.021780,-0.018905,-0.033119
b'CPC020_VCAP_24H:BRD-U88861852-000-01-2:10',-0.021543,0.012827,0.022738,-0.030279,0.018027,0.029957,0.016393,0.015808,-0.026316,0.015436,...,-0.000503,-0.027035,0.074229,-0.046279,-0.100855,0.018087,-0.046194,0.027555,-0.018881,0.033423
b'CPC008_MCF7_24H:BRD-U88878891-000-01-9:10',-0.001908,-0.010882,0.009547,-0.030209,-0.031052,-0.004272,0.032694,0.014692,0.002802,0.007572,...,0.008878,0.039414,-0.010854,-0.005820,0.016359,0.014666,0.007906,-0.057677,-0.002049,-0.015508
b'CPC006_THP1_6H:BRD-U94846492-000-01-1:10',0.016440,0.004836,-0.001130,-0.037635,-0.013796,-0.014536,0.021741,0.012166,-0.046451,-0.015344,...,0.026765,-0.026921,0.034285,0.014385,-0.003842,-0.051430,0.010880,-0.004290,-0.032282,-0.018934
b'CPC014_VCAP_24H:BRD-U97083655-000-01-1:10',0.034109,0.021524,-0.009855,-0.074793,-0.021633,0.001860,0.038829,0.034173,-0.018114,0.027023,...,-0.002615,-0.002455,0.015112,0.024586,-0.076027,-0.034279,0.075895,0.015140,-0.001985,0.042518


In [146]:
for rid in df.index:
    df.loc[rid,'pert_id'] = row_meta_df_g.loc[rid,'pert_id']
    
df['pert_id'] = df['pert_id'].apply(lambda x: x.strip("b'"))

In [147]:
df.set_index('pert_id', inplace = True)

In [150]:
pcid_pertid.set_index('pert_id', inplace = True)
for pertid in df.index:
    df.loc[pertid,'PCID'] = pcid_pertid.loc[pertid,'pubchem_cid']
df = df.dropna(subset = ["PCID"])
df.set_index('PCID', inplace = True)

In [ ]:
df.shape

In [152]:
columns = list(pd.Series(df.columns).apply(lambda x: x.strip("b'")))
df.columns = columns

In [157]:
df = df.T
df.index.name = 'Sig'

In [158]:
df.head()

PCID,6842999,4043357,2853908,44825297,107867,92151,16746329,5702095,66368,46783268,...,23662354,73707392,51371447,6473767,6410757,73707355,73707432,73707391,73707397,MLS003116075
Sig,,,,,,,,,,,,,,,,,,,,,
200814_at,0.006782,-0.003336,-0.031396,0.006320,-0.044147,-0.005684,-0.002450,0.010897,-0.010852,-0.012641,...,0.010626,0.011954,0.025112,-0.005214,-0.018986,0.010751,-0.002023,-0.001820,0.025473,0.003918
222103_at,-0.030192,-0.010254,0.034964,0.006809,0.006812,0.006061,0.035679,0.005570,-0.007054,0.015038,...,0.019743,0.010140,0.013992,0.025741,-0.016572,-0.000078,-0.016592,-0.005512,0.020580,-0.011812
201453_x_at,0.007706,0.009914,0.025250,-0.001783,0.001371,-0.012433,-0.004015,-0.018304,-0.009285,0.000589,...,0.007390,0.019062,0.024848,-0.011731,-0.004154,-0.001234,0.015349,0.007454,-0.003050,-0.007861
204131_s_at,-0.021802,-0.005688,0.034566,0.013459,0.006440,0.009918,-0.008107,-0.010646,0.006271,-0.025098,...,-0.005880,-0.014068,0.056664,-0.004372,-0.013712,0.000392,0.033789,0.014774,-0.018674,-0.024525
200059_s_at,-0.010061,-0.001724,-0.011199,-0.002413,0.010818,-0.004066,0.001781,-0.006065,-0.001289,0.004659,...,0.005372,0.002287,-0.007069,-0.000353,0.012310,0.007356,-0.022608,0.023248,-0.027331,0.010060


## Save Matrix

In [159]:
filename = 'Output/L1000_signatures_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
df.to_csv(filename, sep='\t', compression='gzip')

## Convert Binary Matrix to GMT and Save as GMT

In [137]:
drugset_library = []
for index, row in df.iterrows():
    drugset = []
    drugset.append(index)
    drugset.append('')
    for i in range(df.shape[1]):
        if abs(row.iloc[i])>0.05:
            drugset.append(row.index[i])
#     if len(drugset) >= 3:
    drugset_library.append(drugset)
dsl = np.array(drugset_library)

In [139]:
len(max(dsl, key=len))

128

In [140]:
filename = 'Output/L1000_signatures_DrugSetLibrary_%s.gmt'% str(datetime.date.today())[0:7].replace('-', '_')
with open(filename,'w',encoding='utf-8') as f:
    for row in dsl:
        np.savetxt(f, [row], fmt = '%s',delimiter = '\t')